<a href="https://colab.research.google.com/github/rsalbertiaustral/labo2025v/blob/main/gridsearchexpandido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Seteo del ambiente en Google Colab**

Esta parte se debe correr con el runtime en Python3
Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type -> Python 3

Conectar la virtual machine donde esta corriendo Google Colab con el Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo


los siguientes comando estan en shell script de Linux

**Crear las carpetas en el Google Drive**
"instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria kaggle de Python
Bajar el dataset_pequeno al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab

In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi

limpio el ambiente de R

In [1]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,657245,35.2,1454460,77.7,1326128,70.9
Vcells,1220272,9.4,8388608,64.0,1975128,15.1


In [2]:
options(warn = -1) # Desactiva mensajes de advertencia

In [2]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")
if (!require("primes")) install.packages("primes")
require("primes")

Loading required package: data.table

Loading required package: rpart

Loading required package: parallel

Loading required package: primes

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘primes’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: primes



Aqui debe poner SU semiila primigenia

In [ ]:
PARAM <- list()
# reemplazar por su primer semilla
PARAM$semilla_primigenia <- 163733
PARAM$qsemillas <- 1

PARAM$training_pct <- 70L  # entre  1L y 99L

# elegir SU dataset comentando/ descomentando
PARAM$dataset_nom <- "~/datasets/dataset_pequeno.csv"

In [ ]:
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)
#   crea una particion 70, 30

particionar <- function(data, division, agrupa = "", campo = "fold", start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from = start, length.out = length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
}


In [ ]:
ArbolEstimarGanancia <- function(semilla, training_pct, param_basicos) {
  # particiono estratificadamente el dataset
  particionar(dataset,
    division = c(training_pct, 100L -training_pct),
    agrupa = "clase_ternaria",
    seed = semilla # aqui se usa SU semilla
  )

  # genero el modelo
  # predecir clase_ternaria a partir del resto
  modelo <- rpart("clase_ternaria ~ .",
    data = dataset[fold == 1], # fold==1  es training,  el 70% de los datos
    xval = 0,
    control = param_basicos
  ) # aqui van los parametros del arbol

  # aplico el modelo a los datos de testing
  prediccion <- predict(modelo, # el modelo que genere recien
    dataset[fold == 2], # fold==2  es testing, el 30% de los datos
    type = "prob"
  ) # type= "prob"  es que devuelva la probabilidad

  # prediccion es una matriz con TRES columnas,
  #  llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  # cada columna es el vector de probabilidades


  # calculo la ganancia en testing  qu es fold==2
  ganancia_test <- dataset[
    fold == 2,
    sum(ifelse(prediccion[, "BAJA+2"] > 0.025,
      ifelse(clase_ternaria == "BAJA+2", 117000, -3000),
      0
    ))
  ]

  # escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada <- ganancia_test / (( 100 - PARAM$training_pct ) / 100 )

  return(
    c( list("semilla" = semilla),
      param_basicos,
      list( "ganancia_test" = ganancia_test_normalizada )
     )
  )
}

In [ ]:
ArbolesMontecarlo <- function(semillas, param_basicos) {
  # limitar cantidad de núcleos usados ( ejemplo máximo 4)
  ncores <- min(4, detectCores())

  salida <- mcmapply(ArbolEstimarGanancia,
    semillas,
    MoreArgs = list(PARAM$training_pct, param_basicos),
    SIMPLIFY = FALSE,
    mc.cores = ncores
  )

  return(salida)
}


In [ ]:
# carpeta de trabajo
# por favor cambiar numero de experimento si se cambia el loop principal
setwd("/content/buckets/b1/exp")
experimento <- "HT2900"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [ ]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# trabajo solo con los datos con clase, es decir 202107
dataset <- dataset[clase_ternaria != ""]

In [ ]:
# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, PARAM$qsemillas )

In [ ]:
# genero la data.table donde van los resultados detallados del Grid Search Expandido
# un registro para cada combinacion de < semilla, parametros >

if(file.exists("gridsearch_detalle_EXP2.txt")){
  tb_grid_search_detalle <- fread("gridsearch_detalle_EXP2.txt")
}else{
  tb_grid_search_detalle <- data.table(
    semilla = integer(),
    cp = numeric(),
    maxdepth = integer(),
   minsplit = integer(),
    minbucket = integer(),
    ganancia_test = numeric()
  )
}

cat("Cantidad de combinaciones previas:", nrow( tb_grid_search_detalle ), "\n")

Cantidad de combinaciones previas: 500 


Esta es la parte del código que usted debe expandir a TODOS los hiperparámetros de rpart,
ya que actualmente apenas recorre maxdepth y minsplit dejando fijos cp=-0.5 y minbucket=5

# Experimento 2 - Grid Search expandido (rpart)
- Se amplía la búsqueda a todos los hiperparámetros principales de rpart:
  - cp
  - maxdepth
  - minsplit
  - minbucket
- Se mantiene la estructura del Experimento 1 para comparabilidad.

In [ ]:
# CONFIGURACIÓN DEL DIRECTORIO DE TRABAJO Y REANUDACIÓN EXP2

# Establezco el directorio de trabajo persistente en Google Drive
setwd("/content/buckets/b1/exp/HT2900")
getwd()  # Confirmar que estás en /content/buckets/b1/exp/HT2900

# Si existe archivo parcial, lo cargo para reanudar
archivo_detalle <- "gridsearch_detalle_EXP2.txt"

if (file.exists(archivo_detalle)) {
  cat(" Archivo parcial encontrado, reanudando...\n")
  tb_grid_search_detalle <- fread(archivo_detalle)
  cat(" Filas ya procesadas:", nrow(tb_grid_search_detalle), "\n")
} else {
  cat(" No hay archivo previo, iniciando desde cero...\n")
  tb_grid_search_detalle <- data.table()
}

# Calculo iteraciones ya realizadas (para continuar)
iter <- nrow(tb_grid_search_detalle) / PARAM$qsemillas
if (iter > 0) cat(" Continuando desde iter =", iter, "\n")


[1] "/content/.drive/My Drive/labo1/exp/HT2900"

 Archivo parcial encontrado, reanudando...
 Filas ya procesadas: 500 
 Continuando desde iter = 500 


In [ ]:
# BLOQUE EXPANDIDO DE GRID SEARCH - EXPERIMENTO 2

iter <- 0

# Definición de los grids para cada hiperparámetro
for (vcp in c(-0.5, -0.1, 0.0, 0.001, 0.01)) {            # complejidad mínima
  for (vmax_depth in c(4, 6, 8, 10, 12, 14)) {            # profundidad máxima
    for (vmin_split in c(1000, 800, 600, 400, 200, 100, 50, 20, 10)) {  # mínimo para dividir
      for (vmin_bucket in c(5, 10, 20, 30, 50)) {         # mínimo en hojas


        # Ejecución del modelo
        iter <- iter + 1
        cat("Iter:", iter, "\n")
        flush.console()

        # Evita recalcular combinaciones ya procesadas
        if (iter * PARAM$qsemillas < nrow(tb_grid_search_detalle) + 1) next

        # Defino la lista de hiperparámetros
        param_basicos <- list(
          "cp" = vcp,
          "maxdepth" = vmax_depth,
          "minsplit" = vmin_split,
          "minbucket" = vmin_bucket
        )

        # Ejecuto Montecarlo con las semillas definidas en PARAM
        ganancias <- ArbolesMontecarlo(PARAM$semillas, param_basicos)

        # Agrego resultados parciales a la tabla
        tb_grid_search_detalle <- rbindlist(
          list(tb_grid_search_detalle, rbindlist(ganancias))
        )

        # Guardado parcial cada 50 iteraciones
        if (iter %% 50 == 0) {
          fwrite(tb_grid_search_detalle,
                 file = "gridsearch_detalle_EXP2.txt",
                 sep = "\t")
          cat(" Backup parcial en iter =", iter,
              "| Filas acumuladas:", nrow(tb_grid_search_detalle), "\n")
        }

      }
    }
  }
}

Iter: 1 
Iter: 2 
Iter: 3 
Iter: 4 
Iter: 5 
Iter: 6 
Iter: 7 
Iter: 8 
Iter: 9 
Iter: 10 
Iter: 11 
Iter: 12 
Iter: 13 
Iter: 14 
Iter: 15 
Iter: 16 
Iter: 17 
Iter: 18 
Iter: 19 
Iter: 20 
Iter: 21 
Iter: 22 
Iter: 23 
Iter: 24 
Iter: 25 
Iter: 26 
Iter: 27 
Iter: 28 
Iter: 29 
Iter: 30 
Iter: 31 
Iter: 32 
Iter: 33 
Iter: 34 
Iter: 35 
Iter: 36 
Iter: 37 
Iter: 38 
Iter: 39 
Iter: 40 
Iter: 41 
Iter: 42 
Iter: 43 
Iter: 44 
Iter: 45 
Iter: 46 
Iter: 47 
Iter: 48 
Iter: 49 
Iter: 50 
Iter: 51 
Iter: 52 
Iter: 53 
Iter: 54 
Iter: 55 
Iter: 56 
Iter: 57 
Iter: 58 
Iter: 59 
Iter: 60 
Iter: 61 
Iter: 62 
Iter: 63 
Iter: 64 
Iter: 65 
Iter: 66 
Iter: 67 
Iter: 68 
Iter: 69 
Iter: 70 
Iter: 71 
Iter: 72 
Iter: 73 
Iter: 74 
Iter: 75 
Iter: 76 
Iter: 77 
Iter: 78 
Iter: 79 
Iter: 80 
Iter: 81 
Iter: 82 
Iter: 83 
Iter: 84 
Iter: 85 
Iter: 86 
Iter: 87 
Iter: 88 
Iter: 89 
Iter: 90 
Iter: 91 
Iter: 92 
Iter: 93 
Iter: 94 
Iter: 95 
Iter: 96 
Iter: 97 
Iter: 98 
Iter: 99 
Iter: 100 
Iter: 10

In [ ]:
# Guardado completo al finalizar cada cp
fwrite(tb_grid_search_detalle,
       file = "gridsearch_detalle_EXP2.txt",
       sep = "\t")

cat("\n Guardado total para cp =", vcp,
    "| Filas acumuladas:", nrow(tb_grid_search_detalle), "\n")



 Guardado total para cp = 0.01 | Filas acumuladas: 1350 


In [ ]:
# Final
cat("\n Grid Search EXP2 finalizado. Total filas:",
    nrow(tb_grid_search_detalle), "\n")


 Grid Search EXP2 finalizado. Total filas: 1350 


In [ ]:
# muestro la tabla
tb_grid_search_detalle

semilla,cp,maxdepth,minsplit,minbucket,ganancia_test
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
681041,-0.5,4,1000,5,43640000
681041,-0.5,4,1000,10,43640000
681041,-0.5,4,1000,20,43640000
681041,-0.5,4,1000,30,43640000
681041,-0.5,4,1000,50,47840000
681041,-0.5,4,800,5,43640000
681041,-0.5,4,800,10,43640000
681041,-0.5,4,800,20,43640000
681041,-0.5,4,800,30,43640000


In [ ]:
# genero y grabo el resumen
tb_grid_search <- tb_grid_search_detalle[,
  .(ganancia_mean = mean(ganancia_test, na.rm = TRUE),
  ganancia_sd = sd(ganancia_test, na.rm = TRUE),
  qty = .N),
  by = .(cp, maxdepth, minsplit, minbucket)
   ]

In [ ]:
# ordeno descendente por ganancia
setorder( tb_grid_search, -ganancia_mean )

In [ ]:
# veo los 10 mejores hiperparámetros
tb_grid_search[1:10]

cp,maxdepth,minsplit,minbucket,ganancia_mean,ganancia_sd,qty
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
-0.5,8,600,10,49850000,NA,1
-0.5,8,400,10,49850000,NA,1
-0.5,8,200,10,49850000,NA,1
-0.1,8,600,10,49850000,NA,1
-0.1,8,400,10,49850000,NA,1
-0.1,8,200,10,49850000,NA,1
-0.5,8,50,10,49530000,NA,1
-0.1,8,50,10,49530000,NA,1
-0.5,8,100,10,49430000,NA,1


In [ ]:
# genero un id a la tabla
tb_grid_search[, id := .I ]

fwrite( tb_grid_search,
  file = "gridsearch_resumen_EXP2.txt",
  sep = "\t"
)

In [3]:
# Montar Google Drive en Colab (versión compatible)
library(googledrive)

# fuerza a usar tu cuenta de Google y guarda el token
options(gargle_oauth_email = TRUE)

# autenticar y cachear las credenciales
drive_auth(cache = TRUE)

# establecer el directorio donde trabajás
setwd("/content/buckets/b1/exp/HT2900")

# mostrar los archivos disponibles para confirmar conexión
list.files()


Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=5100962e0230f9a127b4138d94d9679c&access_type=offline&prompt=consent



Enter authorization code: eyJjb2RlIjoiNC8wQVZHelIxQXZOdXJnTV90dTg2WXZPWHBCc3ZSOXdrejJBWXF1cWtNR2drbkRtWDFwYmFibWlRUC01RUc5akxBaUJvUDBMdyIsInN0YXRlIjoiZjlmZWMwZjI2MjdjNzIwYThkNTdlMTQ4ODdjYmEzNmEifQ==


ERROR: [1m[33mError[39m in `drive_auth()`:[22m
[1m[22m[33m![39m Can't get Google credentials.
[36mℹ[39m Are you running [34mgoogledrive[39m in a non-interactive session? Consider:
[36m•[39m Call `drive_deauth()` to prevent the attempt to get credentials.
[36m•[39m Call `drive_auth()` directly with all necessary specifics.
[36mℹ[39m See gargle's "Non-interactive auth" vignette for more details:
[36mℹ[39m [3m[34m<https://gargle.r-lib.org/articles/non-interactive-auth.html>[39m[23m
